The goal of this Notebook is to use the results of the [Stack Overflow 2022 Developer Survey](https://insights.stackoverflow.com/survey) to train a regression neural network for predicting total compensation among programmers. This notebook can be thought of being divided into three different sections: data exploration and visualization, data preprocessing, and the implementation of the neural net. 

In [6]:
import time
import itertools
import math
import warnings
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
try:
  from category_encoders import *
except ModuleNotFoundError:
  ! pip install category_encoders
  from category_encoders import *
import torch
from torch.optim import Adam

In [ ]:
# Data from https://insights.stackoverflow.com/survey year is 2022

try:
  survey = pd.read_csv("survey_results_public.csv")
except FileNotFoundError:
  ! gdown 1T1WH1mZh9xg4b7dmt3mk2mZrvXgzocvS # survey_results_public.csv
  survey = pd.read_csv("survey_results_public.csv")

In [ ]:
from google.colab import files
uploaded = files.upload()
survey = pd.read_csv("survey_results_public.csv")

In [ ]:
def print_percent_this_value(data, feature, value):
  number = data[data[feature] == value].shape[0]
  ratio = 100 * number / data[feature].size
  print(f"{number} records, or {ratio}%, have value {value} for feature {feature}")

In [ ]:
def print_percent_not_nan(data, feature):
  number = data[feature].notna().sum()
  ratio = round(100 * number / data[feature].size, 2)
  print(f"{number} records, or {ratio}%, have a non-NaN value for feature {feature}")

In [ ]:
def get_unique_values_including (dataframe, feature, string):
  uniques = dataframe[feature].unique()
  result = ()
  for response in uniques:
    if string in str(response):
      result.append(response)
  return result

In [ ]:
# For columns that include a mix of checkboxes, this will return the individual checkboxes, as opposed to each unique combination of them
def get_atoms(dataset, field):
  uniques = dataset[field].unique()
  atoms = []
  for unique in uniques:
    if unique is np.NaN:
      continue
    asList = unique.split(';')
    for token in asList:
      toAdd = token.strip()
      if toAdd not in atoms:
        atoms.append(toAdd)
  return atoms

In [ ]:
#Deprecated
def make_piechart(dataset, feature, dicttext):
  copy = dataset.dropna(subset = [feature], inplace = False)
  explode_size = 0.05
  counts = []
  atoms = get_atoms(copy, feature)
  for atom in atoms:
    counts.append(copy[copy[feature].str.contains(atom)].shape[0])
  agedict = dict(zip(atoms, counts))
  explode = [explode_size for index in range(len(counts))]
  fig, ax = plt.subplots(figsize = (30, 8))
  ax.pie(counts, labels=atoms, autopct = '%1.1f%%', explode = explode, shadow = True)
  print(dicttext)
  print(agedict)

In [ ]:
# Makes a bar graph of a categorical feature
def make_bar_graph(dataset, feature, title, xlabel, ylabel, log = False):
  atoms = get_atoms(dataset, feature)
  copy = dataset.dropna(subset = [feature], inplace = False)
  counts = []
  if np.NaN in atoms:
    atoms.remove(np.NaN)
  for atom in atoms:
    if 'Other' in atom:
      counts.append(copy[copy[feature].str.contains('Other')].shape[0])
    else:
      counts.append(copy[copy[feature].str.contains(atom)].shape[0])
  fig, ax = plt.subplots(figsize = (30, 8))
  ourdict = dict(zip(counts, atoms))
  sorted_keys = sorted(ourdict.keys(), reverse = True)
  labels = []
  for key in sorted_keys:
    labels.append(ourdict.get(key))
  pps = ax.bar(x = labels, height = sorted_keys, log = log, align = 'center')
  ax.set_xlabel(xlabel)
  ax.set_xticklabels(labels, rotation=90)
  ax.set_title(title)
  ax.set_ylabel(ylabel)
  # Thanks to https://www.tutorialspoint.com/adding-value-labels-on-a-matplotlib-bar-chart
  for p in pps:
   height = p.get_height()
   ax.annotate('{}%'.format(round(100.0 * height / copy.shape[0], 1)),
      xy=(p.get_x() + p.get_width() / 2, height),
      xytext=(0, 3), # 3 points vertical offset
      textcoords="offset points",
      ha='center', va='bottom')

In [ ]:
#Deprecated and kept in the case it's needed later
def gender_piechart(bracket):
  large_explode = 0.2
  small_explode = 0.5
  counts = (bracket[0].shape[0], bracket[1].shape[0], bracket[2].shape[0], bracket[3].shape[0])
  labels = ('Male', 'Female', 'Nonbinary, Genderqueer, or Gender-Nonconforming', 'Prefer not to Say')
  explode = (large_explode, small_explode, small_explode, small_explode)
  fig, ax = plt.subplots(figsize = (30, 8))
  ax.pie(counts, labels=labels, autopct = '%1.1f%%', shadow = True, explode=explode)

In [ ]:
#Deprecated and kept in the case it's needed later
def age_piechart(dataset):
  explode_size = 0.05
  agecounts = []
  for unique in dataset['Age'].unique():
    agecounts.append(dataset[dataset['Age'] == unique].shape[0])
  agecounts[-1] = dataset['Age'].isna().sum() #Doesn't pick up NaN values, doing these manually
  labels = ('25-34', '35-44', '18-24', '45-54', '55-65', '65+', 'Didn\'t say', '<18', 'No Response')
  agedict = dict(zip(labels, agecounts))
  explode = [explode_size for index in range(len(agecounts) - 4)]
  fig, ax = plt.subplots(figsize = (30, 8))
  ax.pie(agecounts[:-4], labels=labels[:-4], autopct = '%1.1f%%', explode = explode, shadow = True)
  print('The dictionary of each age group and the number of respondents is as follows:\n', agedict)

In [ ]:
# Shows how many years of experience the  respondents tend to have
# Has to be separate from generic bar graph function since the data is quantitative
def workexp_bar_graph(dataframe, title):
  feature = 'YearsCodePro'
  copy = dataframe.dropna(subset = [feature], inplace = False)
  uniques = copy[feature].unique()
  counts = []
  for atom in uniques:
      counts.append(copy[copy[feature] == atom].shape[0])
  fig, ax = plt.subplots(figsize = (30, 8))
  ourdict = dict(zip(counts, uniques))
  sorted_keys = sorted(ourdict.keys(), reverse = True)
  labels = []
  for key in sorted_keys:
    labels.append(ourdict.get(key))
  pps = ax.bar(x = labels, height = sorted_keys, align = 'center')
  ax.set_xlabel('Work Experience in Years')
  ax.set_xticklabels(labels, rotation=90)
  ax.set_title(title)
  ax.set_ylabel('Number of Respondents')
  # Many thanks to https://www.tutorialspoint.com/adding-value-labels-on-a-matplotlib-bar-chart
  for p in pps:
   height = p.get_height()
   ax.annotate('{}%'.format(round(100.0 * height / copy.shape[0], 1)),
      xy=(p.get_x() + p.get_width() / 2, height),
      xytext=(0, 3), # 3 points vertical offset
      textcoords="offset points",
      ha='center', va='bottom')

In [ ]:
pd.set_option("max_columns", None)
target = 'ConvertedCompYearly'
survey.sample(3)

#Data Exploration

The survey features people paid at different intervals and in different currencies. What a nightmare to process! Thankfully, we're given a feature known as ConvertedCompYearly. As it says on the website: 

`We converted salaries from user currencies to USD using the exchange rate on May 24, 2022 and also converted to annual salaries assuming 12 working months and 50 working weeks. `

This is perfect for us! But how many records actually have data for this feature?

In [ ]:

def valid_targets(dataframe):
  print_percent_not_nan(dataframe, "ConvertedCompYearly")
  median = dataframe["ConvertedCompYearly"].median()
  mean = round(dataframe["ConvertedCompYearly"].mean(), 2)
  print(f"the median total compensation is ${round(median / 1000, 2)}k, the average total compensation is ${round(mean / 1000, 2)}k")

In [ ]:
norm = survey.dropna(subset = [target]).copy()
valid_targets(survey)
norm.isna().sum() / (0.01 * norm.shape[0])

##Delete Irrelevant Features

When it comes to feature selection, this model will be pruning down as many features as possible to keep the neural net lean. Someof the names don't describe what the feature describes very well, but there's luckily a copy of the survey provided with the data. The following are some features that can confidently be removed: 

*   ResponseId is an ID number
*   SurveyEase and SurveyLength are about how the person felt about the survey itself
*   CompFreq, CompTotal, and Currency are all rolled into the target variable
*   Blockchain opinions, thoughts on certain technologies, Stack Overflow use, and personal OS are unlikely to matter
*   'Knowledge' refers to encountering knowledge silos and likely doesn't matter
*   Frequency, TimeSearch, and TimeAnswering are also related to SO itself
*   TrueFalse features are extremely unclear as to what it is even with a copy of the survey
*   BuyNewTool refers to how they would decide on buying a new tool












In [ ]:
# Warning, in-place!
def delete_irrelevant_columns(dataframe):
  oldSize = dataframe.shape[1]
  del dataframe["ResponseId"] 
  del dataframe['SurveyEase'] 
  del dataframe['SurveyLength']
  del dataframe['CompFreq'] 
  del dataframe['CompTotal']
  del dataframe['Currency']
  del dataframe['Blockchain']
  del dataframe['BuyNewTool']
  del dataframe['TBranch']
  for col in dataframe:
    if 'WantToWorkWith' in col or 'Personal' in col or 'SO' in col or 'Frequency' in col or 'Knowledge' in col or 'Time' in col or 'TrueFalse' in col: 
      del dataframe[col]
  newSize = dataframe.shape[1]
  print(f"went from {oldSize} to {newSize} features")
  print(f"size of the Dataframe is now {dataframe.shape}")

##Delete Highly-Null Features

How many of the features are null? `VCHostingProfessional use` is literally entirely null. `LearnCodeCoursesCert` is over halfway null, while `LearnCodeOnline`, `MiscTechHaveWorkedWith`, `ProfessionalTech`, and  `Onboarding` are all over 30% null and `PlatformHaveWorkedWith`, `WebframeHaveWorkedWith`, `OfficeStackAsyncHaveWorkedWith`, `ICorPM`, and `WorkExp` are all over 20% null.

In [ ]:
delete_irrelevant_columns(norm)
norm.isna().sum() / (0.01 * norm.shape[0])

In [ ]:
#Warning! In place!
def delete_high_nulls(dataframe):
  oldSize = dataframe.shape[1]
  del dataframe['VCHostingProfessional use']
  del dataframe['LearnCodeCoursesCert']
  del dataframe['LearnCodeOnline']
  del dataframe['PlatformHaveWorkedWith']
  del dataframe['WebframeHaveWorkedWith']
  del dataframe['MiscTechHaveWorkedWith']
  del dataframe['ToolsTechHaveWorkedWith']
  del dataframe['OfficeStackAsyncHaveWorkedWith']
  del dataframe['ICorPM']
  del dataframe['Onboarding']
  del dataframe['ProfessionalTech']
  newSize = dataframe.shape[1]
  print(f"went from {oldSize} to {newSize} features")
  print(f"size of the Dataframe is now {dataframe.shape}")

In [ ]:
delete_high_nulls(norm)
norm.isna().sum() / (0.01 * norm.shape[0])

##Delete Redundant Work Experience Features

`WorkExp` and `YearsCodePro` are over 90% correlated, while `YearsCode` and `YearsCodePro` are also 90% correlated. Of these, `WorkExp` correlates more closely to the target variable of these features, but is also highly null at 28%. Thus, we will be using `YearsCodePro` among these instead, since it has better correlation to the target variable than `YearsCode` and is less than 1% null.

In [ ]:
def experience_correlations(dataframe):
  compcopy = dataframe.copy()
  compcopy.dropna(subset = ['WorkExp', 'YearsCodePro', 'YearsCode'], inplace = True)
  compcopy['YearsCodePro'].replace({'Less than 1 year':0, 'More than 50 years':51 }, inplace = True)
  compcopy['YearsCode'].replace({'Less than 1 year':0, 'More than 50 years':51 }, inplace = True)
  compcopy = compcopy.astype({'YearsCodePro':int, 'YearsCode':int})
  print(compcopy.corr())

In [ ]:
experience_correlations(norm)

Interestingly enough, if we're to only consider those who make a very large amount of money, these features all have a weak *negative* correlation to the target variable.

In [ ]:
def upper_exp_corrs(dataframe):
  high_target = 550_000
  experience_correlations(dataframe[dataframe[target] > high_target])

upper_exp_corrs(norm)

In [ ]:
def delete_years_cols(dataframe):
  oldSize = dataframe.shape[1]
  del dataframe['YearsCode']
  del dataframe['WorkExp']
  newSize = dataframe.shape[1]
  print(f"went from {oldSize} to {newSize} features")
  print(f"size of the Dataframe is now {dataframe.shape}")

In [ ]:
delete_years_cols(norm)
norm.isna().sum() / (0.01 * norm.shape[0])

##Discretionarily Deleted Features

Lastly, to help keep the features as lean as possible, the following features are discretionarily removed.

In [ ]:
# Deleting some of the borderline features that might or might not help predict
def delete_unlikely_cols(data):
  oldSize = data.shape[1]
  del data['RemoteWork']
  del data['Employment']
  del data['LearnCode']
  del data['PurchaseInfluence']
  del data['DatabaseHaveWorkedWith']
  del data['NEWCollabToolsHaveWorkedWith']
  del data['OpSysProfessional use']
  del data['VersionControlSystem']
  del data['VCInteraction']
  del data['OfficeStackSyncHaveWorkedWith']

  newSize = data.shape[1]
  print(f"went from {oldSize} to {newSize} features")
  print(f"size of the Dataframe is now {data.shape}")

In [ ]:
delete_unlikely_cols(norm)
norm.isna().sum() / (0.01 * norm.shape[0])

#Data Visualization

##Overall Total Compensation Visualization

Visualizing the target variable, the distribution is quite heavily weighted toward \$55k or so, with an average of just over \$170k.

In [ ]:
def visualize_salary (dataset, title):
  maxSalary = 550_000
  stepNum = 101
  scaleFactor = 1_000
  scaleSuffix = 'k'
  graphBins = np.linspace(0, maxSalary/scaleFactor, stepNum)
  fig, ax = plt.subplots(figsize=(20, 5))
  ax.hist(dataset[target] / scaleFactor, bins = graphBins)
  ax.set_xlabel('TC in thousands, bin size = $' + str(int(maxSalary / (stepNum - 1))))
  ax.set_ylabel('Number of Respondents')
  percent = 100 * dataset[dataset[target] < maxSalary].shape[0] / dataset[target].shape[0]
  percent = round(percent, 2)
  outsideRange = str(percent) + "% of respondents make within $" + str(int(maxSalary / scaleFactor)) + scaleSuffix
  average = "The average total compensation is $" + str(round(dataset[target].mean() / scaleFactor, 2)) + scaleSuffix
  median = "The median total compensation is $" + str(round(dataset[target].median() / scaleFactor, 2)) + scaleSuffix
  ax.annotate(outsideRange, xy = (0,0), textcoords = 'figure fraction', xytext = (0.7, 0.7), fontsize = 'large')
  ax.annotate(average, xy = (0,0), textcoords = 'figure fraction', xytext = (0.7, 0.6), fontsize = 'large')
  ax.annotate(median, xy = (0,0), textcoords = 'figure fraction', xytext = (0.7, 0.5), fontsize = 'large')
  ax.set_title(title)
  dummy = np.NaN

In [ ]:
america = (norm[norm['Country'] == 'United States of America'])
visualize_salary(norm, 'Total Compensation Distribution for All Respondents')
visualize_salary(america, 'Total Compensation Distribution for American Respondents')

From hereon out, we'll use `norm` to refer to respondants whose total compensation was less than or equal to $550,000 for the upper TC range, with `upper` exclusively holding those records above this cutoff.

In [ ]:
high_target = 550_000
upper = norm[norm[target] > high_target].copy().sort_values(by = target)
norm = norm[norm[target] <= high_target].copy()
upper.sample(3)

##Important Notice

For all the following categorical variables, the respondents were able to pick multiple choices from an array of options. For exampe, this means the same respondent could identify as male and female, heterosexual and bisexual, as a front end developer and a data scientist, etc. In determing the distribution of these attributes, the figures shown are for the amount of respondents who *included* that option in their answer. Someone who identified as both male and female would show up in both the male salary graph and the female salary graph.

##Visualization by Gender

###Pruning

Since the 'In your own words' option doesn't save what the respondent elaborated with for privacy reasons, we will drop this option from each respondent's selection. If this results in an empty Gender field, we'll change it to NaN.

In [ ]:
#Warning! In-place!
def prune_gender(dataframe):
  print('Before pruning \'in your own words\' responses')
  print(dataframe['Gender'].unique())
  dataframe['Gender'].replace(inplace = True, regex = True, to_replace = ['Or, in your own words:;', 'Or, in your own words:'], value = '')
  dataframe['Gender'].replace(inplace = True, to_replace = 'Man;', value = 'Man') #Special case
  dataframe['Gender'].replace(inplace = True, to_replace = '', value = np.NaN)
  print('\n\n\nAfter pruning')
  print(dataframe['Gender'].unique())
  print('\n\n\nDropping ' + str(dataframe['Gender'].isna().sum()) + ' null values for the Gender field out of ' + str(dataframe['Gender'].shape[0]))
  dataframe.dropna(subset = ['Gender'], inplace = True)

  men = dataframe[dataframe['Gender'].str.contains('Man')]
  women = dataframe[dataframe['Gender'].str.contains('Woman')]
  nb = dataframe[dataframe['Gender'].str.contains('Non-binary')]
  undisclosed = dataframe[dataframe['Gender'].str.contains('Prefer not to say')]
  return men, women, nb, undisclosed


In [ ]:
print('Pruning the main dataset')
norm_men, norm_women, norm_nb, norm_und = prune_gender(norm)
print('Pruning the upper earners')
upper_men, upper_women, upper_nb, upper_und = prune_gender(upper)
norms = (norm_men, norm_women, norm_nb, norm_und)
uppers = (upper_men, upper_women, upper_nb, upper_und)

In [ ]:
def graph_by_gender(dataset, title, color, maxSalary, print_range = False, minSalary = 0, scaleSuffix = 'k', scaleFactor = 1_000):
  stepNum = 101
  graphBins = np.linspace(minSalary/scaleFactor, maxSalary/scaleFactor, stepNum)
  fig, ax = plt.subplots(figsize = (20, 5))
  ax.set_xlabel('Total Compensation in thousands, bin size = $' + str(int((maxSalary - minSalary) / (stepNum - 1))), fontsize = 'large')
  ax.set_ylabel('Number of Respondents', fontsize = 'large')
  ax.set_title(title, fontsize = 'large')
  percent = 100 * dataset[dataset[target] <= maxSalary].shape[0] / dataset[target].shape[0]
  percent = round(percent, 2)
  outsideRange = str(percent) + "% of this subset make within " + str(int(maxSalary / scaleFactor)) + scaleSuffix
  average = "The average total compensation is $" + str(round(dataset[target].mean() / scaleFactor, 2)) + scaleSuffix
  median = "The median total compensation is $" + str(round(dataset[target].median() / scaleFactor, 2)) + scaleSuffix
  debug = []
  if (print_range):
    ax.annotate(outsideRange, xy = (graphBins[0],0), textcoords = 'figure fraction', xytext = (0.725, 0.7), fontsize = 'large')
  ax.annotate(average, xy = (graphBins[0],0), textcoords = 'figure fraction', xytext = (0.725, 0.6), fontsize = 'large')
  ax.annotate(median, xy = (graphBins[0],0), textcoords = 'figure fraction', xytext = (0.725, 0.5), fontsize = 'large')
  ax.hist(dataset[target] / scaleFactor, bins = graphBins, color = color)

###Total Compensation by Gender

When considering those who make less than \$550k, the TC distribution follows a similar shape amongst the genders, but the average TC of the male group was 6.4% higher than the average TC for the female group. When looking at those making above $550k, the tables turn; women make 13.2% more than the men within this earnings bracket. Based on these figures, high-earning women are out-earning their male peers.

Concerning the sample size, it should be said that there were only 56 women who reported making above \$550k in the survey, while there were 1391 such men. And while this sample size is low, the proportion of women to men among this earnings cohort is only one percent point lower than among the under $550k cohort (more on this next).

Lastly, these figures are only applicable when dividing solely by gender. There was no further dilineation among country, years of professional experience, age, ethnicity, etc.

####Norm Dataset

In [ ]:
graph_by_gender(norm_men, 'Total Compensation for Men (<=$' + str(int(high_target / 1000)) + 'k )', 'royalblue', high_target)

In [ ]:
graph_by_gender(norm_women, 'Total Compensation for Women (<=$' + str(int(high_target / 1000)) + 'k )', 'crimson', high_target)


In [ ]:
graph_by_gender(norm_nb, 'Total Compensation for Nonbinary, Genderqueer, or Gender-Nonconforming Respondents (<=$' + str(int(high_target / 1000)) + 'k )', 'Purple', high_target)

In [ ]:
graph_by_gender(norm_und, 'Total Compensation for those who Chose not to Disclose Gender (<=$' + str(int(high_target / 1000)) + 'k )', 'olive', high_target)

####Upper Dataset

In [ ]:
graph_by_gender(upper_men, 'SalarTotal Compensation for Men (>$' + str(int(high_target / 1000)) + 'k )', 'royalblue', maxSalary = 5_000_000, minSalary = 500_000, print_range = True)
print(upper_men.shape[0])

In [ ]:
graph_by_gender(upper_women, 'Total Compensation for Women (>$' + str(int(high_target / 1000)) + 'k )', 'crimson', maxSalary = 5_000_000, minSalary = 500_000, print_range = True)
print(upper_women.shape[0])

In [ ]:
graph_by_gender(upper_nb, 'Total Compensation for Nonbinary, Genderqueer, or Gender-Nonconforming Respondents (>$' + str(int(high_target / 1000)) + 'k )', 'Purple', maxSalary = 5_000_000, minSalary = 500_000, print_range = True)

In [ ]:
graph_by_gender(upper_und, 'Total Compensation for those who Chose not to Disclose Gender (>$' + str(int(high_target / 1000)) + 'k )', 'olive', maxSalary = 5_000_000, minSalary = 500_000, print_range = True)

###Gender Distribution

The survey respondents are overwhelmingly male. The gender distribution is essentially identically between the two earnings cohorts: when comparing the ratio of men to women between the two earnings cohorts, women are 5.12% of men among those making less than $550k, and 4.06% of men among those making more. 

In [ ]:
make_bar_graph(norm, 'Gender', 'Gender in the Norm dataset',  'Gender',  'Count')

In [ ]:
make_bar_graph(upper, 'Gender', 'Gender in the Upper Dataset',  'Gender',  'Count')

##Age Distribution

Almost half each dataset is between 25 and 34 years old, with  over 70% of each dataset being 25 and 44 years old. The upper dataset is ever so slightly older.

In [ ]:
make_bar_graph(norm, 'Age', 'Age among Lower - Earnings Respondents',  'Age Bracket',  'Count')

In [ ]:
make_bar_graph(upper, 'Age', 'Age among Upper - Earnings Respondents',  'Age Bracket',  'Count')

##Sexuality Distribution

Each dataset has an essentially identical distribution by sexuality. 87% of both datasets identified as heterosexual.

In [ ]:
make_bar_graph(norm, 'Sexuality', 'Sexuality in Norm Dataset',  'Sexuality',  'Count')

In [ ]:
make_bar_graph(upper, 'Sexuality', 'Sexuality in Upper Dataset',  'Sexuality',  'Count')

In [ ]:
make_bar_graph(norm, 'Trans', 'Transgender Status among Respondents', 'Transgender Status', 'Count')

##Transgender Distriubtion

96-97% of respondents identified as not being transgender across both datasets.

In [ ]:
make_bar_graph(norm, 'Trans', 'Transgender Status in Norm Dataset', 'Transgender Status', 'Count')

In [ ]:
make_bar_graph(upper, 'Trans', 'Transgender Status in Upper Dataset', 'Transgender Status', 'Count')

##Ethnicity Distribution and Total Compensation by Ethnicity

There were lots of options the respondent could've picked between when it comes to ethnicity. And, like with gender, the respondent is able to pick as many options as they feel apply. With so many individual options to represent, making a graph for each one is a tall order. Instead, the dataset will be split into different dataframes of respondents whose ethnicity *includes* each individual option, and the mean and median total compensation for each will be displayed. Remember, the survey respondents come from across the world and have their respective currencies converted into USD, and features like gender aren't dilineated.

In [ ]:
def ethnicity_combos(dataframe):
  uniques = dataframe['Ethnicity'].unique()
  atoms = get_atoms(dataframe, 'Ethnicity')
  print(f'There are a total of {len(uniques)} unique combinations chosen from {len(atoms)} individual options')

In [ ]:
ethnicity_combos(norm)

In [ ]:
ethnicity_combos(upper)

In [ ]:
def salary_by_ethnicity(dataframe):
  atoms = get_atoms(dataframe, 'Ethnicity')
  noNaNs = dataframe.dropna(subset = ['Ethnicity'], inplace = False)
  for atom in atoms:
    if 'Indigenous' in atom: #Special case, is 0 otherwise somehow
      atom = 'Indigenous'
    ethnicity = noNaNs[noNaNs['Ethnicity'].str.contains(atom)]
    number = ethnicity.shape[0]
    percent = (round(100.0 * number / noNaNs.shape[0], 2))
    print(f'{number} respondents include {atom} among their ethnicities, or {percent}%')
    print(f'The average total compensation among this ethnicity is ${round(ethnicity[target].mean() / 1000, 2)}k, and the median is ${round(ethnicity[target].median() / 1000, 2)}k\n')

In [ ]:
salary_by_ethnicity(norm)

In [ ]:
salary_by_ethnicity(upper)

##DevType Distribution

Across both datasets, the most popular roles were front end, back end, and full stack. There aren't any significant differences in the distributions of developer type between the two datasets.

Remember, since one person could pick multiple titles, the sum of these percentages won't be 100%! 

In [ ]:
make_bar_graph(norm, 'DevType', 'Distribution of Developer Titles', 'Title of Respondent', 'Count')

In [ ]:
make_bar_graph(upper, 'DevType', 'Distribution of Developer Titles', 'Title of Respondent', 'Count')

##Work Experience Distribtion

The most frequent number of years worked for each dataset is 3 years, at about 8% of respondents for both. However, for the next 3 most common responses to the question, the upper earners have been in the field a bit longer. The average for the 2nd-4th most frequent responses in the norm dataset is 2.67 years, while the average for the upper dataset is 7 years.

In [ ]:
workexp_bar_graph(norm, 'Work Experience for Lower Earnings Cohort')

In [ ]:
workexp_bar_graph(upper, 'Work Experience for Higher Earnings Cohort')

# Data Preprocessing

##Encoding

As with most datasets, we arrive at the issue of encoding. Simple one-hot encoding results in such a monstrous amount of features, Google Colab runs out of RAM and interrupts execution. Even if we were to one-hot encode along the atoms of each choice and have them be indepententally applicable, instead of making a new feature for each unique combination of atoms selected, this could add dozens and dozens of new features per categorical feature. Going from about 20 features to hundreds of features is clearly not the greatest for dimensionality!

Enter [target encoding](https://sci-hub.se/10.1145/507533.507538) (a less academic explanation is [here](https://maxhalford.github.io/blog/target-encoding/)\). Essentially, each category within a feature is replaced by a blend of the mean target variable among all records that have that category in that feature  and the overall mean target.

For example, if someone indicates their `Sexuality` as bisexual, this categorical string is replaced by a blend of the mean total compensation among all those who indicate their sexuality as solely `bisexual`  and the overall total compensation average. If their response was `bisexual;straight`, it would be encoded with a blend of the average total compensation of all those who chose `bisexual;straight` as their sexuality and the average overall total compensation.

Target encoding has the benefit of creating no new features; not a single extra column is added to the data frame. And, as icing on the cake, target encoding can impute missing values as well. If someone skipped the question asking for their `Sexuality`, their `NaN` value would be replaced with a blend of the average total compensation of everyone else who skipped that question and the average overall total compensation.

In [ ]:
def get_x_and_y(dataframe):
  copy = dataframe.copy()
  y = copy.pop(target)
  return copy, y

In [ ]:
#Non-destructive function to return a target-encoded dataframe and its encoder
def sample_target_encoding(dataframe):
  encoder = TargetEncoder()
  train_x, train_y = get_x_and_y(dataframe)
  encoder.fit(train_x, train_y)
  # Some encoders behave differently on whether y is given or not. This is mainly due to regularisation in order to avoid overfitting. 
  # On training data transform should be called with y, on test data without.
  transformed = encoder.transform(train_x, train_y)
  return transformed, encoder, train_y

In [ ]:
#Non-destructive function to return a leave-one-out-encoded dataframe, its encoder, and the target variable
def sample_loo_encoding(dataframe):
  encoder = LeaveOneOutEncoder(sigma = 0.325)
  train_x, train_y = get_x_and_y(dataframe)
  encoder.fit(train_x, train_y)
  # Some encoders behave differently on whether y is given or not. This is mainly due to regularisation in order to avoid overfitting. 
  # On training data transform should be called with y, on test data without.
  transformed = encoder.transform(train_x, train_y)
  return transformed, encoder, train_y

###Target Encoding Example

In [ ]:
norm.head(3)

In [ ]:
transformed_norm, norm_encoder, dummy = sample_target_encoding(norm)
transformed_norm.head(3)

###Leave-One-Out Encoding Example

Leave One Out encoding is essentially target encoding except that the record currently being encoded is not used in the encoding calculation, and the sklearn implementation has support for introducing some Gaussian randomness into the result.

In [ ]:
transformed_norm, norm_encoder, norm_y = sample_loo_encoding(norm)
transformed_norm.head(3)

##Correlations

Now that everything in the dataframe has been encoded and imputed, we can look at correlations. `Ethnicity`, `YearsCodePro`, `Age`, and especially `Country` seem to be the highest predictors among the `norm` dataset, whereas the `upper` dataset doesn't seem to have any strong correlations except for `Country`, which has a weaker correlation of about 0.25 compared to the norm dataset's 0.45

###Norm Correlations

In [ ]:
transformed_norm[target] = norm_y
transformed_norm.corr()

###Upper Correlations

In [ ]:
transformed_upper, upper_encoder, upper_y = sample_loo_encoding(upper)
transformed_upper[target] = upper_y
transformed_upper.corr()

##Scaling and Principle Component Analysis

Before starting with the neural net, we'll create a function that, given a dataframe, will encode it, scale it, and perform PCA on it, returning data_x and data_y as tensors. This will fully preprocess the data and format it in a way that makes running it through the neural net easier.

In [ ]:
# As of now, we are not scaling the target variable.
def scale_data(data_x, data_y):
  scaler = StandardScaler()
  scaler = scaler.fit(data_x)
  result_x = scaler.transform(data_x)
  result_y = data_y
  # Don't scale Y
  return result_x, result_y, scaler

In [ ]:
# Dimensionality reduction
def pca_data(data_x, verbose, variance):
  pca = None
  if variance == -1:
    pca = PCA()
  else:
    pca = PCA(n_components = variance, svd_solver = 'full')
  oldSize = len(data_x[0])
  result = pca.fit_transform(data_x)
  newSize = len(result[0])
  if verbose:
    print(f'Reduced data from {oldSize} features to {newSize} features')
  return result

This is the function we'll use. Given a dataset, an amount of variance to keep during PCA, and whether to print progress messages, the function will return the x data, the y data, and the scaler used 

In [ ]:
# Non-destructive function that uses target encoding, a standard sklearn scaler, and PCA to preprocess a dataframe and return as a tensor_x and tensor_y
def encode_normalize_pca(data, verbose = True, variance = -1): 
  warnings.filterwarnings("ignore")
  if verbose:
    print('Beginning encoding')
  data_x, encoder, data_y = sample_loo_encoding(data)
  if verbose:
    print("Encoding done")
  data_y = data_y.to_numpy()
  if verbose:
    print('Beginning normalization')
  data_x, data_y, scaler = scale_data(data_x, data_y)
  if verbose:
    print('Normalization done')
    print('Beginning PCA')
  data_x = pca_data(data_x, verbose, variance)
  if verbose:
    print('PCA done\n')
  result_x = torch.tensor(data_x).float()
  result_y = torch.tensor(data_y).float()
  return result_x, result_y, scaler



In [ ]:
#Non-destructive way to convert a whole Pandas dataframe into a data_x and a data_y in tensor form
def get_tensors(data):
  copy = data.copy()
  pandasY = copy.pop(target)
  resultX = torch.tensor(copy.values)
  resultY = torch.tensor(pandasY.values)
  return resultX, resultY

In [ ]:
def root_mean_squared_error(N, yPreds, yReals):
  sum = torch.sum(torch.square(yPreds - yReals))
  result = torch.sqrt(sum / N)
  return result.item()

In [ ]:
# K is the number of features, actual are the real test_y values, and yHats are the predictions
def r2_score(K, actual, yHats):
  N = len(actual)
  yActual = torch.Tensor(actual)  # actual as 1-d Tensor
  yRealMean = torch.mean(yActual)
  yHatMean = torch.mean(yHats)
  numerator = (torch.sum(torch.square(yHats - yHatMean))).item()
  denominator = (torch.sum(torch.square(yActual - yRealMean))).item()
  if denominator == 0:
    print(f'denominator is 0 for R2 score! Returning 1')
    return 1
  rSquared =  numerator / denominator
  numerator = (1 - rSquared) * (N - 1)
  denominator = N - K - 1
  rSquaredAdjusted = 1 - (numerator/denominator)
  return rSquaredAdjusted

In [ ]:
# Deprecated for its inferior performance, but kept around for giggles
def epoch_train(net, train_x, train_y, test_x, test_y, bat_size, epochs, begin, learning_rate, verbose):
  num_prints = 5
  net = net.train()  # Set training mode
  loss_func = torch.nn.MSELoss()  # Mean squared error
  optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
  num_records = len(train_x)
  batches = math.ceil(num_records / bat_size)
  if verbose:
    print(f'Running a total of {batches*epochs} batches and {epochs} epochs')
  for epoch in range(epochs):
    # Shuffle time!
    shuffling = torch.randperm(len(train_x)) # Shuffling key
    epoch_train_x = train_x[shuffling]
    epoch_train_y = train_y[shuffling]
    batches_x = torch.split(epoch_train_x, bat_size) # `N / bat_size` tensors of size `bat_size` x `features` each
    batches_y = torch.split(epoch_train_y, bat_size)
    for batch in range(len(batches_x)):
      optimizer.zero_grad() # "Sets the gradients of all optimized torch.Tensor s to zero." Resets for next grad descent?
      oupt = net(batches_x[batch]) # Forward pass
      loss_obj = loss_func(oupt, batches_y[batch])
      loss_obj.backward()  # Compute gradients
      optimizer.step()     # Update weights and biases, backward pass
    if epoch % (epochs // num_prints) == 0: # prints num_prints times
      currentTime = time.time()
      elapsed = currentTime-begin
      net = net.eval()
      r2, rmse = r2_and_rmse(net, test_x, test_y)
      net = net.train()
      if verbose:
        print("epoch = %6d" % epoch, end="\t")
        print("epoch loss = %7.0f   " % loss_obj.item(), end="\t")
        print(f"R2 = {round(r2, 4)}\tRMSE = ${round(rmse / 1000, 3)}k   ", end = "\t")
        print(f'\tTime elapsed: {int(elapsed // 60)}m {int(elapsed % 60)}s ', end = '\t')
        print(f"{100 * epoch // epochs}% done")
  currentTime = time.time()
  elapsed = currentTime-begin
  if verbose:
    print(f"\nTraining complete. Time elapsed: {int(elapsed // 60)}m {int(elapsed % 60)}s")
  return net

In [ ]:
def get_train_and_test(splits, test_index):
  asList = list(splits)
  test = asList.pop(test_index)
  train = torch.cat(asList)
  return train, test

In [ ]:
def shuffle_datasets(set1, set2):
  if len(set1) != len(set2):
    return None, None
  shuffling = torch.randperm(len(set1)) # Shuffling key
  result1 = set1[shuffling]
  result2 = set2[shuffling]
  return result1, result2

In [ ]:
# Like lower function but also prints result
def eval(net, test_x, test_y):
  r2, rmse = r2_and_rmse(net, test_x, test_y)
  print(f"Model has R2 = {round(r2, 4)} and RMSE = ${round(rmse / 1000, 3)}k\n")
  return r2, rmse

def r2_and_rmse(net, test_x, test_y):
  N = len(test_y)
  net = net.eval()  # set eval mode
  preds = net(torch.Tensor(test_x)).view(N) # all predicted as 1-d Tensor
  # print(f'Making preds for eval, here\'s what we\'ve got\n{preds}')
  # preds = torch.exp(preds)  # Rescale predictions by exp for evaluation
  R2 = r2_score(len(test_x[0]), test_y, preds)
  rmse = root_mean_squared_error(N, preds, test_y)
  absolute_diffs = preds - test_y
  # print(f'\nHow much each prediction overshoots the target: \n{absolute_diffs}')
  # print(f'Overall, the average overshoot is ${round(torch.mean(absolute_diffs).item() / 1000, 3)}k')

  return R2, rmse

#Regression Modeling

##Neural Net Code

The neural net! It has a configurable number of nodes per hidden layer and a configurable number of hidden layers, although always at least one. Each hidden layer will have the same number of nodes, and the model will output a single continuous value. Nodes have their weights initialized according to Xavier intialization and their biases initialized to 0.

In [ ]:
# Will always have at least one hidden layer
class NeuralNet(torch.nn.Module):

  def __init__(self, num_inputs, hidden_layers, hidden_layer_nodes):
    super(NeuralNet, self).__init__()
    self.hiddenLayers = torch.nn.ModuleList()
    firstHidden = torch.nn.Linear(num_inputs, hidden_layer_nodes)
    self.hiddenLayers.append(firstHidden) #maps input to the first hidden layer
    torch.nn.init.xavier_uniform_(self.hiddenLayers[0].weight) # Randomizes weights for this mapping via the Xavier/Glodot intitialization technique
    torch.nn.init.zeros_(self.hiddenLayers[0].bias) # Sets the biases to 0 for this mapping

    for layer in range(1, hidden_layers):  # start at 1 since the initial input layer maps to a hidden layer
      self.hiddenLayers.append(torch.nn.Linear(hidden_layer_nodes, hidden_layer_nodes))
      torch.nn.init.xavier_uniform_(self.hiddenLayers[layer].weight)
      torch.nn.init.zeros_(self.hiddenLayers[layer].bias)
    
    self.hiddenLayers.append(torch.nn.Linear(hidden_layer_nodes, 1)) # Output layer
    torch.nn.init.xavier_uniform_(self.hiddenLayers[-1].weight)
    torch.nn.init.zeros_(self.hiddenLayers[-1].bias)

  def forward(self, x):
    # print(f'{x} is being fed forward')
    activation_func = torch.nn.LeakyReLU()
    z = activation_func(self.hiddenLayers[0](x))
    # print(f'fed forward,\n{x}\n is now {z}\n')
    for layer in range(1, len(self.hiddenLayers) - 1): # Start from 1 since (input->1st_hidden) is done outside of the loop
      z = activation_func(self.hiddenLayers[layer](z))
      # print(f'fed forward, z is now {z}\n')
    # Last layer will be done outside the loop as an identity/ReLU
    lastFunc = torch.nn.Identity()
    z = lastFunc(self.hiddenLayers[-1](z))
    # print(f'last feed forward, output is {z}\n\n')
    return z

I found myself in the curious circumstance that the net would sometimes not train at all over a cross-validation fold. As in, a drop from maybe 69_000 root mean squared error to around 67_000 after maybe 5min in that fold. Any time I have it train for longer, it'll always pretty much stay at whatever RMSE it started at. Some hyperparameter tuning helped or hurt the initial value, but nothing I tweaked resulted in actual learning. 

Hoping to see where the issue is, I slimmed down the test and train set to a single record among the both of them, and something even weirder happens. Sometimes the model will converge and get down to within 1 RMSE in a matter of 100 or 200 epochs. Sometimes, however, the net will only ever spit out a prediction of 0 every time and literally train none at all even on a train and test set made of the same singular record. The RMSE at the end of the full cross validation is literally the exact same as the test_y value. 

I'm most confused since I can run the exact same code back to back, taking a random record each time, and sometimes the model will converge and sometimes it just spits out 0s. Right now the net is just 22 input nodes, a single hidden layer of 14 nodes, and a singular output node. Data_x is encoded and scaled from [0,1] while data_y isn't. Activation functions I've tried include ReLU, ELU, sigmoid, and Tanh. All of them have this problem, but when Sigmoid and Tanh learn it's at a glacial rate (RMSE from 76_000 to 75_469 after 200 epochs and one record). Using Xavier initialization for weights. Before implementing proper cross validation, the net would improve its training accuracy but I'm not sure if the test accuracy has ever improved.

##Cross Validate Model

This function is the main heavy lifter of the entire notebook. Given some hyperparameters and a dataframe, it will return the Root mean Squared Error and the R<sup>2</sup> score across each fold and over the entire cross validation. The process goes like this:

*    If `shuffling` is set to true, the dataframe has the order of its records randomly shuffled
*    The data is split into K folds
*    The neural net object is initialized according to the given hyperparameters
*    The neural net is trained, giving printouts for the model's performance according to the given fold's test set 5 times each fold
     *    If `train_by_epoch` is set to true, the neural net is trained by shuffling the training and test data, feeding in `batch_size` records, and running backpropagation until all records in the dataset have been fed through. This process is repeated `epoch` times
     *    If `train_by_epoch` is set to false, the neural net is trained by feeding in `batch_size` random records then running back propogation a total of `epochs * num_records // batch_size` times
*    At the end of training, the Root Mean Squared Error and R<sup>2</sup> scores are printed and returned

In [ ]:
def cross_validate_model(data, hidden_layers, shuffle = True, train_by_epoch = False, nodes_per_layer = None, bat_size = 10, variance = 0.95, epochs = 1000, partitions = 5, learning_rate = 0.01, verbose = True):
  begin = time.time()
  copy = data.copy()
  features = len(data_x[0])
  hidden_nodes = nodes_per_layer
  if hidden_nodes == None:
    hidden_nodes = (features * 2) // 3 # if nodes per hidden layer unspecified, set to 2/3 the number of features
  # shuffle
  if shuffle:
    if verbose:
      print('Shuffling before cross validation!')
    copy.sample(frac = 1).reset_index(drop = True)
  else:
    if verbose:
      print('No shuffling before cross validation')
  # Split into k folds
  partitions_x = torch.tensor_split(data_x, partitions)
  partitions_y = torch.tensor_split(data_y, partitions)
  r_squared = 0
  rmse = 0
  for partition in range(partitions):
    if verbose:
      print(f'Beginning fold {partition + 1} of {partitions}')
    train_x, test_x = get_train_and_test(partitions_x, partition)
    train_y, test_y = get_train_and_test(partitions_y, partition)
    # if verbose:
    #   train_x = data_x[10:12]
    #   train_y = torch.tensor(data_y[10:12])
    #   test_x = train_x
    #   test_y = train_y
    #   print(f'train_x {train_x}')
    #   print(type(train_x))
    #   print(f'train_y {train_y}')
    #   print(type(train_y))
    # need to preprocess here
    net = NeuralNet(features, hidden_layers, hidden_nodes)
    if train_by_epoch:
      epoch_train(net, train_x, train_y, test_x, test_y, bat_size, epochs, begin, learning_rate, verbose)
    else:
      if verbose:
        print('Training with random sampling')
      net = random_sampling_train(net, train_x, train_y, test_x, test_y, bat_size, epochs, begin, learning_rate, verbose)
    r_squareAdd, rmseAdd = eval(net, test_x, test_y)
    r_squared = r_squared + (r_squareAdd / partitions)
    rmse = rmse + (rmseAdd / partitions)
  currentTime = time.time()
  elapsed = currentTime-begin
  if verbose:
    print(f'Validation complete! ({int(elapsed // 60)}m {int(elapsed % 60)}s elapsed)')
    print(f"Cross-validated model with {partitions} partitions has:\nR2 = {round(r_squared, 4)}\t RMSE = ${round(rmse / 1000, 2)}k")
    print(f'hidden_layers = {hidden_layers}, bat_size = {bat_size}, nodes_per_layer = {hidden_nodes}, variance = {variance}, epochs = {epochs}, learning_rate = {learning_rate}')
  return r_squared, rmse

In [ ]:
def random_sampling_train(net, train_x, train_y, test_x, test_y, bat_size, epochs, begin, learning_rate, verbose):
  # print(f'Here\'s the test set for this fold:\n{test_y}')
  num_prints = 5
  net = net.train()  # Set training mode
  loss_func = torch.nn.MSELoss()  # Mean squared error
  optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
  n_items = len(train_x)
  batches_per_epoch = n_items // bat_size
  max_batches = epochs * batches_per_epoch
  if verbose:
    print(f'Running a total of {max_batches} batches')
  for b in range(max_batches):
    curr_bat = np.random.choice(n_items, bat_size,
      replace=False)
    X = torch.Tensor(train_x[curr_bat])
    Y = torch.Tensor(train_y[curr_bat]).view(bat_size,1)
    # X = torch.Tensor(train_x)
    # Y = Tensor(train_y)
    # Y = torch.log(Y)  # Scale y values down to their log for training
    optimizer.zero_grad()
    oupt = net(X)
    loss_obj = loss_func(oupt, Y)
    loss_obj.backward()
    optimizer.step()
    if b % (max_batches // num_prints) == 0: # prints num_prints times
      currentTime = time.time()
      elapsed = currentTime-begin
      net = net.eval()
      r2, rmse = r2_and_rmse(net, test_x, test_y)
      net = net.train()
      if verbose:
        print("batch = %6d" % b, end="\t")
        print("batch loss = %3.4f   " % loss_obj.item(), end="\t")
        print(f"R2 = {round(r2, 4)}\tRMSE = ${round(rmse / 1000, 3)}k   ", end = "\t")
        print(f'\tTime elapsed: {int(elapsed // 60)}m {int(elapsed % 60)}s ', end = '\t')
        print(f"{100 * b // max_batches}% done")
  currentTime = time.time()
  elapsed = currentTime-begin
  if verbose:
    print(f"\nTraining complete. Time elapsed: {int(elapsed // 60)}m {int(elapsed % 60)}s")
  return net

In [ ]:
norm_hispanic  = norm.dropna(subset = ['Ethnicity'])
norm_hispanic =  norm_hispanic[norm_hispanic['Ethnicity'].str.contains('Hispanic')]
norm_america =  norm[norm['Country'] == 'United States of America'].copy()
america_jobs = norm_america.dropna(subset = ['DevType'], inplace = False)
print(america_jobs.shape)
america_ds = america_jobs[america_jobs['DevType'].str.contains('Data scientist')].copy()
america_ds = america_ds[america_ds[target] > 60_000].copy()
america_ds.shape

In [ ]:
hidden_layers = 3
nodes_per_layer = None
bat_size = 20
epochs = 1_000
variance = 0.95
partitions = 5
learning_rate = 0.01

rsquare, rmse = cross_validate_model(america_ds, hidden_layers = hidden_layers, nodes_per_layer = nodes_per_layer, bat_size = bat_size, epochs = epochs, variance = variance, partitions = partitions)

Two features were tested: shuffling the training and test data before partitioning, and randomly sampling `number_records / batch_size` batches from the data  vs sequentially taking `batch_size` records from the training set. The results are as  follows, sorted by the best Root Mean Squared Error

*    Shuffling, random sample training Model has R2 = 0.547 and RMSE = \$53.833k
*    No shuffling, random sample training R2 = 0.4835	 RMSE = \$57.1k
*    No shuffling, sequential epoch training Model has R2 = -0.0023 and RMSE = \$73.832k
*      Shuffling before, sequential epoch training Model has R2 = -0.0052 and RMSE = \$74.111k


The best results were with random sampling and a shuffled dataset. Randomly sampling the data makes the biggest difference, and shuffling before partitioning into a training and test dataset also helps a bit on top of that. Shuffling doesn't make much of a difference  when training sequentially through the epoch.

Thus, from now on, we will be shuffling before partitioning and randomly sampling records from the data.

Next steps:
*    Shuffle data pre-partitioning to see if cross validation loses the appearance of "building up" on the last cross. If that doesn't work, try printing weights upon start and end of each fold
*    Look into validity of R<sup>2</sup> as an evaluation metric for this use case https://statisticsbyjim.com/regression/r-squared-invalid-nonlinear-regression/
*    Look into using other activation functions
*    Look into further dividing the dataset 
*    Research neural nets for use in regression in particular
*    Look into feature engineering and hyperparameter tuning

Using ReLU with learning rate of 0.01, Adams optimizer, batch size of 20, 100 epochs, keeping 95% of the variance during PCA, with 5 Fold cross validation yields a RMSE of about \$56k
* Switching the dataset to not be restricted to America? Boom, 54.59k RMSE
*    LeakyReLU makes very little difference, possibly even slightly worse
*    RReLU about the same, 56.9k
*    GELU about the same, 57k
*    ELU a bit worse, 58.45k
*    PReLU maybe a bit better, 56.43k
*    Adjusting learning rate from 0.01 makes it worse at 0.1, goes from 56k to 58k RMSE, lowering to 0.005 or 0.001 keeps around 56k RMSE

In [ ]:
data_x, data_y, encoder = encode_normalize_pca(norm_america)

In [ ]:
def pare_features(dataframe, features):
  result = pd.DataFrame([])
  for feature in features:
    result[feature] = dataframe[feature]

  return result

In [ ]:
def find_best_features(data, hidden_layers, shuffle = True, train_by_epoch = False, nodes_per_layer = None, bat_size = 10, variance = 0.95, epochs = 1000, partitions = 5, learning_rate = 0.01, verbose = True):
  dataframe = data.copy()
  targets = dataframe.pop(target)
  num_features = dataframe.shape[1]
  columns = dataframe.columns
  bestRMSE = 200_000
  toUse = []
  for to_include in range(2, num_features):
    print(f'Checking for {to_include} features.\nBest RMSE so far: {bestRMSE}')
    print(f'Best features: {toUse}\n')
    subsets = list(itertools.combinations(columns, to_include))
    for subset in subsets:
      newData = pare_features(dataframe, subset)
      newData[target] = targets
      r_squared, rmse = cross_validate_model(data = newData, hidden_layers = hidden_layers, shuffle = shuffle, train_by_epoch = train_by_epoch, nodes_per_layer = nodes_per_layer, bat_size = bat_size, epochs = epochs, variance = variance, partitions = partitions, verbose = verbose)
      if rmse < bestRMSE:
        bestRMSE = rmse
        toUse = subset
      

In [ ]:
hidden_layers = 3
nodes_per_layer = None
bat_size = 20
epochs = 1_000
variance = -1
partitions = 5
learning_rate = 0.01

usable = survey.dropna(subset = [target])
usable = usable[usable[target] <= high_target]
find_best_features(usable, hidden_layers = hidden_layers, verbose = True)

In [ ]:
subsets = list(itertools.combinations(norm.columns, 2))
subset = subsets[0]
newData = pd.DataFrame([])
for feature in subset:
  newData[feature] = norm[feature]
newData.head()